In [1]:
! pip install transformers datasets tokenizers evaluate
! pip install transformers[sentencepiece]
! pip install torch
! pip install tensorflow
! pip install spacy
! pip install seqeval
! pip install ipywidgets
! pip install "ray[tune]" transformers datasets scipy scikit-learn torch

  Using cached evaluate-0.4.0-py3-none-any.whl (81 kB)


  Using cached tensorflow-2.11.0-cp38-cp38-macosx_10_14_x86_64.whl (244.3 MB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached tensorflow_estimator-2.11.0-py2.py3-none-any.whl (439 kB)
  Using cached keras-2.11.0-py2.py3-none-any.whl (1.7 MB)
  Using cached tensorflow_io_gcs_filesystem-0.30.0-cp38-cp38-macosx_10_14_x86_64.whl (1.6 MB)
  Using cached numpy-1.24.2-cp38-cp38-macosx_10_9_x86_64.whl (19.8 MB)
  Using cached absl_py-1.4.0-py3-none-any.whl (126 kB)
  Using cached gast-0.4.0-py3-none-any.whl (9.8 kB)
  Using cached tensorboard-2.11.2-py3-none-any.whl (6.0 MB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached grpcio-1.51.1-cp38-cp38-macosx_10_10_x86_64.whl (4.6 MB)
  Using cached flatbuffers-23.1.21-py2.py3-none-any.whl (26 kB)
  Using cached libclang-15.0.6.1-py2.py3-none-macosx_10_9_x86_64.whl (25.0 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any.whl (781 k

     |████████████████████████████████| 43 kB 1.6 MB/s eta 0:00:01
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16170 sha256=843a743ccef40434ebafdc10b035fe23fec6f14fad6026ac9c35c26e0eba4d86
  Stored in directory: /Users/breaux/Library/Caches/pip/wheels/ad/5c/ba/05fa33fa5855777b7d686e843ec07452f22a66a138e290e732
Successfully built seqeval


     |████████████████████████████████| 76.6 MB 1.7 MB/s eta 0:00:01
  Using cached protobuf-4.21.12-py2.py3-none-any.whl (165 kB)
     |████████████████████████████████| 8.7 MB 2.6 MB/s eta 0:00:01


     |████████████████████████████████| 468 kB 2.6 MB/s eta 0:00:01
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.14.0
    Uninstalling protobuf-3.14.0:
      Successfully uninstalled protobuf-3.14.0
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you'll have protobuf 4.21.12 which is incompatible.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you'll have protobuf 4.21.12 which is incompatible.
cached-path 1.1.2 requires huggingface-hub<0.6.0,>=0.0.12, but you'll have huggingface-hub 0.12.0 which is incompatible.
allennlp 2.9.3 requires torch<1.12.0,>=1.6.0, but you'll have torch 1.13.1 which is incompatible.
allennlp 2.9.3 requires transformers<4.19,>=4

In [3]:
! pip install numpy==1.21

     |████████████████████████████████| 16.9 MB 2.3 MB/s eta 0:00:01
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.2
    Uninstalling numpy-1.24.2:
      Successfully uninstalled numpy-1.24.2
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

torchvision 0.12.0 requires torch==1.11.0, but you'll have torch 1.13.1 which is incompatible.
tensorflow 2.11.0 requires protobuf<3.20,>=3.9.2, but you'll have protobuf 4.21.12 which is incompatible.
tensorboard 2.11.2 requires protobuf<4,>=3.9.2, but you'll have protobuf 4.21.12 which is incompatible.
allennlp 2.9.3 requires torch<1.12.0,>=1.6.0, but you'll have torch 1.13.1 which is incompatible.
allennlp 2.9.3 requires transformers<4.19,>=4.1, but you'll have transformers 4.26

In [2]:
# mount to folder
from google.colab import drive
drive.mount('/content/drive')
%cd /content/drive/MyDrive/mobile_privacy/cleaned/models

Mounted at /content/drive
/content/drive/MyDrive/mobile_privacy/cleaned/models


In [1]:
# imports
import transformers
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline
import sys, os,json
#sys.path.insert(1, '/content/drive/MyDrive/mobile_privacy/cleaned')
import datasets
import transformers
from transformers import DataCollatorForTokenClassification
import evaluate
import numpy as np
from transformers import Trainer
from transformers import TrainingArguments
from ray import tune
from seqeval.metrics import classification_report
from sklearn.preprocessing import MultiLabelBinarizer
RANDOM_SEED = 0

In [2]:
def load_dataset(path, split=[0.9, 0.05, 0.05]):
    '''
    Load the dataset from the path as a Dataset object.
    Dataset format is:
        - id: the id of the scenario
        - tokens: tokennized words
        - ner_tags: the NER tags of the tokens
    @ param path: file path to dataset json file.
    @ param split: the [train, validation, test] split.
    '''
    with open(path, 'r') as f:
        data1 = json.load(f)
    res = {'id': [], 'tokens': [], 'ner_tags': []}
    for id, data in data1.items():
        res['id'].append(id)
        res['tokens'].append(data['words'])
        res['ner_tags'].append(data['codes'])
    dataset = datasets.Dataset.from_dict(res, features=datasets.Features(
                {
                    "id": datasets.Value("string"),
                    "tokens": datasets.Sequence(datasets.Value("string")),
                    "ner_tags": datasets.Sequence(
                        datasets.features.ClassLabel(
                            names=[
                                'O', 'B-SIM', 'I-SIM', 'B-COM', 'I-COM', 'B-QUE', 'I-QUE'
                            ]
                        )
                    ),
                }
            ))
    if len(split) != 3 or abs(split[0] + split[1] + split[2] - 1.0) > 1e-5:
      print(split)
      raise Exception("split must be in the form [train, validation, test], and the split should sum up to 1")
    train_testvalid = dataset.shuffle(seed=RANDOM_SEED).train_test_split(test_size=split[1] + split[2], seed=RANDOM_SEED)
    # Split the 10% test + valid in half test, half valid
    test_valid = train_testvalid['test'].train_test_split(test_size=split[2]/(split[1] + split[2]), seed=RANDOM_SEED)
    # gather everyone if you want to have a single DatasetDict
    train_test_valid_dataset = datasets.dataset_dict.DatasetDict(
        {
        'train': train_testvalid['train'],
        'validation': test_valid['train'],
        'test': test_valid['test']
        }
    )
    return train_test_valid_dataset

In [3]:
# tokenize and align dataset
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

def tokenize_and_align_labels(examples, **kwargs):
    '''
    Input: a row of Dataset
    Output: use dataset.map() method to map this function for each row.
            This will tokenize each row and align the original labels.
    '''
    tokenizer = kwargs['tokenizer']
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
    tokenized_inputs["labels"] = new_labels
    return tokenized_inputs

In [4]:
# execution codes
# 1. load dataset
# change DATASET_PATH to your dataset path
DATASET_PATH = '../datasets/scenarios-labeled.json'
raw_datasets = load_dataset(DATASET_PATH, [0.8, 0.1, 0.1])
ner_feature = raw_datasets["train"].features["ner_tags"]
label_names = ner_feature.feature.names

# 2. auto tokenizer
model_checkpoint = "dslim/bert-base-NER"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
tokenized_datasets = raw_datasets.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=raw_datasets["train"].column_names,
    fn_kwargs={'tokenizer':tokenizer}
)

# 3. data collator and metric
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)
metric = evaluate.load("seqeval")
def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    MultiLabelBinarizer().fit_transform(true_labels)
    MultiLabelBinarizer().fit_transform(true_predictions)
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    # return all_metrics
    print(classification_report(true_labels,true_predictions ))
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

# 4. define model
id2label = {i: label for i, label in enumerate(label_names)}
label2id = {v: k for k, v in id2label.items()}

def model_init():
    return AutoModelForTokenClassification.from_pretrained(
            model_checkpoint,
            id2label=id2label,
            label2id=label2id,
            ignore_mismatched_sizes=True
        )
    
# 5. train
args = TrainingArguments(
    "bert-finetuned-ner",
    evaluation_strategy="epoch",
    save_strategy="epoch",
    learning_rate=2e-5,
    num_train_epochs=10,
    weight_decay=0.01,
    push_to_hub=False,
)
trainer = Trainer(
    # model=model,
    model_init = model_init,
    args=args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

trainer.train()

KeyError: '-'